In [2]:
import os
from utils.Preprocessing import filter,features,plots
import pandas as pd

In [3]:
df=pd.read_csv('Organized Data/china/abb_china_36501_statistics.csv')

In [ ]:
import pandas as pd
import numpy as np

# Assuming df is your DataFrame and 'HOUR' is your time column
df['HOUR'] = pd.to_datetime(df['HOUR'])

# Selecting features
features_considered = ['CT', 'TAV', 'SHOT_COUNT']
features = df[features_considered]

# Standardizing the features
dataset = features.values
data_mean = dataset.mean(axis=0)
data_std = dataset.std(axis=0)
dataset = (dataset-data_mean)/data_std

# Creating the sliding window
def create_dataset(X, y, time_steps=1):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        v = X.iloc[i:(i + time_steps)].values
        Xs.append(v)
        ys.append(y.iloc[i + time_steps])
    return np.array(Xs), np.array(ys)

time_steps = 30

# reshape to [samples, time_steps, n_features]
X_train, y_train = create_dataset(df[features_considered], df['PHASE'], time_steps)
print(X_train.shape, y_train.shape)

In [ ]:
import pandas as pd
import numpy as np

# Assuming df is your DataFrame and 'HOUR' is your time column
df['HOUR'] = pd.to_datetime(df['HOUR'])
df = df.sort_values('HOUR')

# Selecting features
features_considered = ['CT', 'TAV', 'SHOT_COUNT']

# Creating the sliding window
def create_dataset(X, time_steps=1):
    Xs = []
    for i in range(len(X) - time_steps):
        v = X.iloc[i:(i + time_steps)].values
        Xs.append(v[..., np.newaxis])  # Adding an extra dimension for each feature
    return np.array(Xs)

time_steps = 30

# reshape to [samples, time_steps, n_features]
for feature in features_considered:
    X_train = create_dataset(df[feature], time_steps)
    print(f"{feature} shape: {X_train.shape}")

print(X_train)

In [ ]:
import pandas as pd
import numpy as np

# Assuming df is your DataFrame and 'HOUR' is your time column
df['HOUR'] = pd.to_datetime(df['HOUR'])
df = df.sort_values('HOUR')

# Selecting features
features_considered = ['CT', 'TAV', 'SHOT_COUNT']

# Creating the sliding window
def create_dataset(X, time_steps=1):
    Xs = []
    for i in range(len(X) - time_steps):
        v = X.iloc[i:(i + time_steps)].values
        Xs.append(v)
    return np.array(Xs)

time_steps = 30

# reshape to [samples, time_steps, n_features]
X_train = create_dataset(df[features_considered], time_steps)
print(f"Tensor shape: {X_train.shape}")

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# Assuming df is your DataFrame and 'HOUR' is your time column
df['HOUR'] = pd.to_datetime(df['HOUR'])
df = df.sort_values('HOUR')

# Selecting features
features_considered = ['CT', 'TAV', 'SHOT_COUNT']

# One-hot encoding the 'PHASE' feature
y = pd.get_dummies(df['PHASE']).values

# Creating the sliding window
def create_dataset(X, y, time_steps=1):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        v = X.iloc[i:(i + time_steps)].values
        Xs.append(v)
        ys.append(y[i + time_steps])
    return np.array(Xs), np.array(ys)

time_steps = 720  # Represents 30 days of hourly data

# reshape to [samples, time_steps, n_features]
X, y = create_dataset(df[features_considered], y, time_steps)

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Training set shape: {X_train.shape}, {y_train.shape}")
print(f"Testing set shape: {X_test.shape}, {y_test.shape}")


In [ ]:
print(X_train)

In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# Assuming df is your DataFrame and 'HOUR' is your time column
df['HOUR'] = pd.to_datetime(df['HOUR'])
df = df.sort_values('HOUR')

# Selecting features
features_considered = ['CT', 'TAV', 'SHOT_COUNT']

# One-hot encoding the 'PHASE' feature
y = pd.get_dummies(df['PHASE']).values

# Creating the sliding window
def create_dataset(X, y, time_steps=1):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        v = X.iloc[i:(i + time_steps)].values
        Xs.append(v)
        ys.append(y[i:i + time_steps])
    return np.array(Xs), np.array(ys)


time_steps = 720  # Represents 30 days of hourly data

# reshape to [samples, time_steps, n_features]
X, y = create_dataset(df[features_considered], y, time_steps)

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Training set shape: {X_train.shape}, {y_train.shape}")
print(f"Testing set shape: {X_test.shape}, {y_test.shape}")

Training set shape: (12664, 720, 3), (12664, 720, 4)
Testing set shape: (3167, 720, 3), (3167, 720, 4)


In [ ]:
# # Define the desired proportion for each phase in the testing set
# # For example, if you want 25% of each phase in the testing set, you can set it to 0.25
# testing_proportion = 0.25

# # Create a dictionary to keep track of how many samples from each phase are included in the testing set
# phase_count = {phase: int(len(y) * testing_proportion) for phase in range(y.shape[1])}

# # Lists to store indices for the training and testing sets
# train_indices, test_indices = [], []

# # Shuffle the data while ensuring that each phase is represented in both sets
# for i in range(len(X)):
#     phase = np.argmax(y[i])
#     if phase_count[phase] > 0:
#         test_indices.append(i)
#         phase_count[phase] -= 1
#     else:
#         train_indices.append(i)

# # Split the data into training and testing sets
# X_train = X[train_indices]
# X_test = X[test_indices]
# y_train = y[train_indices]
# y_test = y[test_indices]
# print(y_train)

In [5]:
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from keras.layers import TimeDistributed

# Assuming X_train, y_train, X_test, y_test are your training and testing sets
n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], y_train.shape[2]

# Function to create the model
def create_model(optimizer='adam'):
    model = Sequential()
    model.add(LSTM(50, return_sequences=True, input_shape=(n_timesteps,n_features)))
    model.add(TimeDistributed(Dense(n_outputs, activation='softmax')))
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model
# Function to fine-tune hyperparameters
def fine_tune_model(X_train, y_train):
    model = KerasClassifier(build_fn=create_model, verbose=0)
    # define the grid search parameters
    optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
    param_grid = dict(optimizer=optimizer)
    grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
    grid_result = grid.fit(X_train, y_train)
    # summarize results
    print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
    means = grid_result.cv_results_['mean_test_score']
    stds = grid_result.cv_results_['std_test_score']
    params = grid_result.cv_results_['params']
    for mean, stdev, param in zip(means, stds, params):
        print("%f (%f) with: %r" % (mean, stdev, param))

# Create the LSTM model
model = create_model()

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=2)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test, verbose=2)
print(f"Loss: {loss}, Accuracy: {accuracy}")

# Fine-tune hyperparameters
# fine_tune_model(X_train, y_train)


Epoch 1/10
396/396 - 74s - loss: 1.2160 - accuracy: 0.4640
Epoch 2/10
396/396 - 68s - loss: 1.1077 - accuracy: 0.5082
Epoch 3/10
396/396 - 77s - loss: 1.0383 - accuracy: 0.5443
Epoch 4/10
396/396 - 80s - loss: 0.9725 - accuracy: 0.5720
Epoch 5/10
396/396 - 68s - loss: 0.9182 - accuracy: 0.5915
Epoch 6/10
396/396 - 71s - loss: 0.8735 - accuracy: 0.6097
Epoch 7/10
396/396 - 67s - loss: 0.8454 - accuracy: 0.6302
Epoch 8/10
396/396 - 71s - loss: 0.7673 - accuracy: 0.6733
Epoch 9/10
396/396 - 73s - loss: 0.7195 - accuracy: 0.6950
Epoch 10/10
396/396 - 68s - loss: 0.6825 - accuracy: 0.7178
99/99 - 5s - loss: 0.6464 - accuracy: 0.7368
Loss: 0.6463587880134583, Accuracy: 0.7368022799491882


In [6]:
model.save('abb_china_36501.h5')

In [12]:
df = pd.read_csv('Organized Data/germany/abb_germany_225414_statistics.csv')
# Selecting features
features_considered = ['CT', 'TAV', 'SHOT_COUNT']

# One-hot encoding the 'PHASE' feature
y = pd.get_dummies(df['PHASE']).values

# Creating the sliding window
def create_dataset(X, y, time_steps=1):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        v = X.iloc[i:(i + time_steps)].values
        Xs.append(v)
        ys.append(y[i:i + time_steps])
    return np.array(Xs), np.array(ys)


time_steps = 720  # Represents 30 days of hourly data

# reshape to [samples, time_steps, n_features]
X, y = create_dataset(df[features_considered], y, time_steps)

loss, accuracy = model.evaluate(X, y, verbose=2)
print(f"Loss: {loss}, Accuracy: {accuracy}")

496/496 - 28s - loss: 3.1848 - accuracy: 0.2599
Loss: 3.184824228286743, Accuracy: 0.2598792016506195


In [13]:
X

array([[[  0., 273., 105.],
        [  0., 274.,   0.],
        [  0., 275.,   0.],
        ...,
        [146., 363., 246.],
        [146., 362., 246.],
        [147., 361., 246.]],

       [[  0., 274.,   0.],
        [  0., 275.,   0.],
        [  0., 276.,   0.],
        ...,
        [146., 362., 246.],
        [147., 361., 246.],
        [146., 362., 246.]],

       [[  0., 275.,   0.],
        [  0., 276.,   0.],
        [  0., 274.,   0.],
        ...,
        [147., 361., 246.],
        [146., 362., 246.],
        [146., 363., 245.]],

       ...,

       [[  0., 290.,   0.],
        [  0., 284.,   0.],
        [  0., 282.,   0.],
        ...,
        [  0., 299.,   0.],
        [  0., 296.,   0.],
        [  0., 296.,   0.]],

       [[  0., 284.,   0.],
        [  0., 282.,   0.],
        [  0., 283.,   0.],
        ...,
        [  0., 296.,   0.],
        [  0., 296.,   0.],
        [  0., 292.,   0.]],

       [[  0., 282.,   0.],
        [  0., 283.,   0.],
        [  0., 2

In [ ]:

df=filter(df)
df

In [ ]:
# tav_arr,sc_arr,ct_arr=features(df)


In [ ]:
plots(df)